# HP Tuning in neural network

In this notebook we will learn hyperparameter tuning using Randomsearch method.
Data used is downloaded from UCI Repository from the following link.

Ref data: https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope
        
I have only used a sample set of original data for learning purpose. 
You can download the full csv from the website if you are interested.

In [1]:
# Importing necesary packages
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV 
# if you are getting an error by importing RandomSearchCV (which is shown here), try check .
# from sklearn.grid_search import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


Tune for epochs and batch size

In [3]:
# load the dataset 
data = pd.read_csv(r'...\magic_gamma_telescope04_sample.csv')

In [4]:
#viewing data
data.head()

,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [5]:
data['class'].value_counts()

g    2183
h    2104
Name: class, dtype: int64

In [6]:
data['class']=data['class'].map({"g":1,"h":0})
# the two response classes "g" and "h" is to be encoded for modeling purpose.
data.head(5)

,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1


In [7]:
# split into predictors and response
X = data.iloc[:,0:-1] #predictors
Y = data.iloc[:,-1] #response

In [8]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(8, input_dim=10, activation='relu'))
    # Single hidden layer with 8 neurons is used.
    mymodel.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [9]:
# create model
model = KerasClassifier(build_fn=create_my_model)

In [10]:
# define the grid search parameters
# This determines the search space combinations.
batchSize = [4,12]
epochs = [3, 5, 8]

In [11]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs)
mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3, verbose=3)
grid_result = mygrid.fit(X, Y)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/5
4287/4287 [==============================] - 1s 178us/step - loss: 50.4305 - accuracy: 0.5724
Epoch 2/5
4287/4287 [==============================] - 1s 134us/step - loss: 3.5377 - accuracy: 0.6702
Epoch 3/5
4287/4287 [==============================] - 1s 133us/step - loss: 1.7256 - accuracy: 0.6688
Epoch 4/5
4287/4287 [==============================] - 1s 136us/step - loss: 1.0882 - accuracy: 0.6930
Epoch 5/5
4287/4287 [==============================] - 1s 138us/step - loss: 0.8175 - accuracy: 0.7089


In [12]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.624446 using {'batch_size': 12, 'epochs': 5}


Activation functions can also be optmized as follows

In [14]:
# create model
model = KerasClassifier(build_fn=create_my_model, epochs=5, batch_size=12)

In [15]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam'] # Some of the available optimizers.
parameter_grid = dict(optimizer=optimizer)

In [16]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

Epoch 1/5
4287/4287 [==============================] - 1s 185us/step - loss: 5.5277 - accuracy: 0.6947
Epoch 2/5
4287/4287 [==============================] - 1s 137us/step - loss: 2.0992 - accuracy: 0.7103
Epoch 3/5
4287/4287 [==============================] - 1s 126us/step - loss: 0.8706 - accuracy: 0.6968
Epoch 4/5
4287/4287 [==============================] - 1s 171us/step - loss: 0.6445 - accuracy: 0.7091
Epoch 5/5
4287/4287 [==============================] - 1s 133us/step - loss: 0.5678 - accuracy: 0.7371


In [17]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.636576 using {'optimizer': 'RMSprop'}


In this way you can create your own search space with respect to various hyperparameters. 
It all depends upon your application what hyperparameter you need to take in to consideration 
for tuning. Once the best fit hyperparameters are found, you can finalize your model and continue for further experiments
or deployment.